In [292]:
import pandas as pd 
import requests
import json

In [293]:
pd.set_option('display.max_colwidth', None)

In [294]:
COW_GRAPH_URL = "https://api.thegraph.com/subgraphs/name/cowprotocol/cow"
COW_AUCTION_API = "https://api.cow.fi/mainnet/api/v1/solver_competition/by_tx_hash/"

def make_graph_query_request(_query):
    return requests.post(COW_GRAPH_URL, json={'query': _query}).json()

def get_cow_auction(tx_hash):
    return requests.get(COW_AUCTION_API + tx_hash).json()

In [295]:
opps = pd.json_normalize(json.load(open('data/opportunities.json')))
len(opps["cowOrder.id"].unique())

46

In [296]:
query = """
    {
        orders(where: {
            id_in: [%s]
        }) {
            id
            trades(first: 10) {
                timestamp, 
                txHash, 
                settlement {
                    solver { address }
                }
                sellToken {
                    decimals
                }
                buyToken {
                    decimals
                }
                sellAmount	
                buyAmount
                feeAmount
            }
        }
    }""" % ",".join([f'"{i}"' for i in opps["cowOrder.id"].unique()])

graph_response = make_graph_query_request(query)

In [297]:
settlements = []

for order in graph_response["data"]["orders"]:
    for trade in order["trades"]:
        auction = get_cow_auction(trade["txHash"])
        winning_surplus = 0 if auction.get("solutions") is None else max([s["objective"]["surplus"] for s in auction["solutions"]])

        settlements.append({
            "order_id": order["id"],
            "solver": trade["settlement"]["solver"]["address"],
            "sell_amount_fix": (int(trade["sellAmount"])-int(trade["feeAmount"])) / 10**trade["sellToken"]["decimals"],
            "buy_amount_fix": int(trade["buyAmount"]) / 10**trade["buyToken"]["decimals"],
            "winning_surplus": winning_surplus,
            "tx_hash": trade["txHash"],
        })

settlements_df = pd.DataFrame(settlements)
settlements_df

,order_id,solver,sell_amount_fix,buy_amount_fix,winning_surplus,tx_hash
0,0x0235fdf6659eaa3d6273f32972f866bfba297baf7b67d24851566568e8499ad466ea2d335291a5afa8d3522eef6adfc3f8746e16643d4d90,0xc9ec550bea1c64d779124b23a26292cc223327b6,11818.082972,5.703457,2.620104e+16,0x4510799898c7e60d1106b5a5d0458c112a701d67425d45f33678e989ec074f22
1,0x05f7ddbdf798e6baa07342983f1e315a498e46a0953ff3c6e5a9c8e1c7bdef550b5dd26f44d36141d97a3bf6841e7ba63c092af9643d94ab,0x3cee8c7d9b5c8f225a8c36e7d3514e1860309651,1586.443392,0.761862,2.684053e+15,0xfada5e97917a391192314a2df1aeda43fcce8f1b6b37da431a9b6e8064f27363
2,0x0b77e604f4791a37605d2e3b807e3dd0dfbe7d8ac27cdedb88797c4c0480ac2989ecb99d52f9c0c7d3aa574178278bcc3276c965643d7501,0x3cee8c7d9b5c8f225a8c36e7d3514e1860309651,483.835431,0.233405,1.769505e+15,0x895312eaa300339c2e96e62e6143daff9d7db62143390d253dd7f80f3168d8f4
3,0x0de23176a4cb67a88bbe54674648ee03e1a853e8ca80eef1b1b416284df163a589ecb99d52f9c0c7d3aa574178278bcc3276c965643ecdaa,0x149d0f9282333681ee41d30589824b2798e9fb47,1966.293645,0.937974,0.000000e+00,0xc2cc712a2bf73824c08858f0264fb97a8f691c77940c8eeacc9127d5b5c14f65
4,0x10af43fbda2b36a55ea8a96741aa96929e1db7af7a3d012126244069774a269dc5281ac3fb899342b39fc8d406759ba1dff846c2643dcdaf,0xc9ec550bea1c64d779124b23a26292cc223327b6,1044.106984,0.502765,3.052806e+15,0x210ff4e48745aac2e90f7e5dc7dcd28e3e099b0fde37bcfc072cd6d6bdc73dff
5,0x116b2a4aac4e82ac7673243a8e1df1d4f922714f8f855c58b99001b6bf0c56d7d361dfdd3249aa67a758668ec2ce6afee8962e55643db41c,0xa21740833858985e4d801533a808786d3647fb83,2.492718,5182.876382,2.775047e+16,0x604a29cba95c5538821bfc91bc41a8572ed2b63507b641836685b15bd04e21f1
6,0x172b177104265feb9343668ce76454458c7e1cb09c74fb928f4bf881aebce9d137c8afca3946c5c5e75c73b44c1f6c0f5bef3a7f643d90a0,0xa21740833858985e4d801533a808786d3647fb83,249.577629,0.120000,1.534098e+17,0x3c01a773e02748ee186350fff89db645136ecc587871b7b85078df355c0fb7a7
7,0x1799016a31f055527fd67ebf8740bf5bbe2a0d759636c3edb5a5c98961ac61c740a50cf069e992aa4536211b23f286ef88752187ffffffff,0xc9ec550bea1c64d779124b23a26292cc223327b6,0.032851,69.791908,2.871499e+17,0x79c196c262a9d9d59197ffe7abde7e86d5c9419070f7544361cd4920d8571260
8,0x191732e117d6278a12c249d08868fb6b40bca28d42696d4a42d40b760bc37db9a334d9bb907240fe01fa4a1d8baeec4cc0a75747643db9ea,0xbff9a1b539516f9e20c7b621163e676949959a66,0.118273,3490.000000,1.482479e+16,0x7d21adcec803cf756292cd7346d2faede0a53887df41236e881678196410fa90
9,0x1b017dd9fe89fb94e1cb9f305c96c2bf3778b802297db6f14404de4aa2836d43f074830329534ccc1ede18cacfbe87efada0853b643d5432,0xc9ec550bea1c64d779124b23a26292cc223327b6,25.997233,0.013926,1.249988e+16,0x6dcfe2dddc0a52740fa7f198c6cfe5b47a14a841ffd2a16848c693624092bfb2


### Positive spread

In [298]:
def get_duration_sec(series):
    return int((series.max()-series.min()) / 1e3)



opps_with_settlements = opps.merge(settlements_df, left_on="cowOrder.id", right_on="order_id", how="inner")
prices = opps_with_settlements['binanceOrderSpreadEth']/(opps_with_settlements["cowOrder.amountIn"] - opps_with_settlements["binanceMatch.amountIn"] + opps_with_settlements["binanceMatch.amountOut"] - opps_with_settlements["cowOrder.amountOut"])
opps_with_settlements["spread_binance_settlement"] = ((opps_with_settlements["sell_amount_fix"] - opps_with_settlements["binanceMatch.amountIn"]) + (opps_with_settlements["binanceMatch.amountOut"] - opps_with_settlements["buy_amount_fix"]))*prices
opps_with_settlements["spread_binance_order"] = opps_with_settlements["binanceOrderSpreadEth"]
opps_with_settlements["spread_binance_oneinch"] = opps_with_settlements["binanceOneinchSpreadEth"]
opps_with_settlements["winning_surplus"] /= 1e18

# groupby order-id and agg: max spread, min spread, count, 
agg_opps = opps_with_settlements \
    .groupby(by="cowOrder.id") \
    .agg(
        spread_binance_settlement_max=("spread_binance_settlement", max),
        spread_binance_settlement_min=("spread_binance_settlement", min),
        spread_binance_order_max=("spread_binance_order", max),
        spread_binance_order_min=("spread_binance_order", min),
        spread_binance_oneinch_max=("spread_binance_oneinch", max),
        spread_binance_oneinch_min=("spread_binance_oneinch", min),
        duration_sec=("timestamp_ms", get_duration_sec),
        winning_surplus=("winning_surplus", max),
    ) \
    .reset_index() \
    .sort_values(by="spread_binance_settlement_max", ascending=False)
agg_opps["have_winning_surplus"] = agg_opps["spread_binance_order_max"] > agg_opps["winning_surplus"]

agg_opps



,cowOrder.id,spread_binance_settlement_max,spread_binance_settlement_min,spread_binance_order_max,spread_binance_order_min,spread_binance_oneinch_max,spread_binance_oneinch_min,duration_sec,winning_surplus,have_winning_surplus
36,0xc4053191dbcbc6d6d91441e5dc2e312db15a67db23af9308a823a36596d866a9fbe87d602f7d7dd511349be4acf58842392124a064444d84,0.055752,-0.001465,0.058557,0.001340,0.074031,-0.013649,275,0.002805,True
19,0x5c1b33929dbd24d765cf4355e56cd021180541089cb0a3843ebaefa8065e1fe5f6f6d531ed0f7fa18cae2c73b21aa853c765c4d8643eaaf3,0.037936,0.029518,0.086087,0.077669,79.777381,62.071316,7,0.067479,True
24,0x7d13d5c41459b5ab8928bea1835a777cf095c9449f740f6ec80417a2e0e963ea6e26a34a8ca973a4a5a41213b79cf2b0cb1a508d643d9754,0.017259,-0.005013,0.146049,0.123776,0.018769,-0.024633,69,0.385994,False
26,0x881cfb08f43b542afe6e9402a3ccf12ea23d36bd63796fcf96ed9935dcc9abdb40a50cf069e992aa4536211b23f286ef88752187ffffffff,0.008085,-0.033910,2.000716,1.958721,21.706215,-65.343092,17,1.992631,True
0,0x0235fdf6659eaa3d6273f32972f866bfba297baf7b67d24851566568e8499ad466ea2d335291a5afa8d3522eef6adfc3f8746e16643d4d90,0.008052,0.000113,0.034247,0.026309,0.008194,0.000113,46,0.026201,True
39,0xd6abfa4b62ee1145dabcfb3752c595d98a1797a67cace9af84d9ec63d341b68509043a2d7ca461f3b76e34cfe082b03417b3f24a643dd3df,0.006197,-0.002049,0.076836,0.068590,0.007067,-0.001179,6,0.070668,True
10,0x1c7daadce9aeead417835b10b721cdc9b01681d76fc414ec7c93f975c7ec9e2209d445b46f9b7af6eb7d45605dced90839654e0d643e9fbc,0.004673,-0.002966,0.074102,0.066463,0.015840,0.008201,7,0.069429,True
27,0x8e7c26170fae40f6d507434e3cceb683fd8b3c0a5d7a1b50dfd3208ba747b4d022f5413c075ccd56d575a54763831c4c27a37bdb643eb22c,0.003294,0.001354,0.028896,0.026957,0.003784,0.001752,17,0.029230,False
34,0xb23b4446d66c6f191664bdf997a127d6cd9dbedd5e8565cd6ace3a51ea8e2660862675dc379dc5fabef945110dbfd58466402661643d5b72,0.002647,-0.002229,0.022735,0.017859,-0.001262,-0.006137,56,0.034912,False
13,0x3a73ea64a4cef94ba2d997da2c2b45678fe4546d1a20093e3fc379cb0b74d5fb0b5dd26f44d36141d97a3bf6841e7ba63c092af9643dd100,0.002511,0.000476,0.018225,0.016189,0.005674,0.003639,3,0.015713,True


In [302]:
agg_opps.loc[(agg_opps.spread_binance_settlement_max > 0) & (agg_opps.have_winning_surplus)]["spread_binance_settlement_max"].sum()

0.12842942044799585

## Simulation 

In [304]:
from collections import defaultdict


best_opps_for_order = opps_with_settlements.loc[opps_with_settlements.groupby("cowOrder.id")["spread_binance_order"].idxmax(axis=0)]
best_opps_for_order.loc[best_opps_for_order.tokenIn == "0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee", "tokenIn"] = "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2"
best_opps_for_order.loc[best_opps_for_order.tokenOut == "0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee", "tokenOut"] = "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2"


starting_balance = 1e6
binance_balance = defaultdict(lambda: starting_balance)
binance_balance_min = defaultdict(lambda: starting_balance)
onchain_balance = defaultdict(lambda: starting_balance)
onchain_balance_min = defaultdict(lambda: starting_balance)

for (i, order) in best_opps_for_order.sort_values(by="timestamp_ms").iterrows():
    binance_amount_in = order["binanceMatch.amountIn"]
    binance_amount_out = order["binanceMatch.amountOut"]
    token_in = order["tokenIn"]
    token_out = order["tokenOut"]

    if binance_balance[token_in] < binance_amount_in:
        print(f"Insufficient balance for {token_in} on binance")
        continue

    binance_balance[token_in] -= binance_amount_in
    binance_balance[token_out] += binance_amount_out

    binance_balance_min[token_in] = min(binance_balance_min[token_in], binance_balance[token_in])
    binance_balance_min[token_out] = min(binance_balance_min[token_out], binance_balance[token_out])

    if order.winning_surplus > order.spread_binance_order:
        # print(f"Order {order['cowOrder.id']} would lose auction")
        continue

    if onchain_balance[token_out] < binance_amount_out:
        print(f"Insufficient balance for {token_out} on chain")
        continue

    onchain_balance[token_out] -= binance_amount_out  # ! suppose we give all surplus to the maker
    onchain_balance[token_in] += binance_amount_in

    onchain_balance_min[token_in] = min(onchain_balance_min[token_in], onchain_balance[token_in])
    onchain_balance_min[token_out] = min(onchain_balance_min[token_out], onchain_balance[token_out])




print("Binance balance deltas")
for token, balance in binance_balance.items():
    print(f"\t{token}: {balance-starting_balance} (max-used: {starting_balance-binance_balance_min[token]})")
print("Onchain balance deltas")
for token, balance in onchain_balance.items():
    print(f"\t{token}: {balance-starting_balance} (max-used: {starting_balance-onchain_balance_min[token]})")

Binance balance deltas
	0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48: 151905.96355938143 (max-used: 1000000.0)
	0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2: -55.84863445512019 (max-used: 1000000.0)
	0x6b175474e89094c44da98b954eedeac495271d0f: -30793.245285494835 (max-used: 1000000.0)
	0xdac17f958d2ee523a2206206994597c13d831ec7: -936.3882890000241 (max-used: 1000000.0)
	0x2260fac5e5542a773aa44fbcfedf7c193bc2c599: -0.11844384472351521 (max-used: 1000000.0)
Onchain balance deltas
	0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2: 100.89603361603804 (max-used: 1000000.0)
	0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48: -235995.90388179326 (max-used: 1000000.0)
	0x6b175474e89094c44da98b954eedeac495271d0f: 25223.333819224034 (max-used: 1000000.0)
